<div style="padding: 10px; background-color: #ecf0f1;">
**<span style="color:red">NOTE:</span>** This notebook requires [`BSpline2.scala`](http://cobweb.cs.uga.edu/~mec/BSpline2.scala) to be placed in the root of your JupyterHub workspace. If you're on `hub.aods.io`, the you can do this using the following AODS Upload link: http://upload.aods.io/http://cobweb.cs.uga.edu/~mec/BSpline2.scala.
</div>

<div style="padding: 10px; background-color: #ecf0f1;">
**<span style="color:red">NOTE:</span>** Commands that begin with a single colon (e.g., <code>:silent</code>) are [Scala REPL commands](https://docs.scala-lang.org/overviews/repl/overview.html). Commands that begin with a double colon (e.g., <code>::relation</code>) are [ScalaTion Kernel commands](https://github.com/scalation/scalation_kernel/blob/master/USER.md). Anything else in a code cell is assumed to be Scala code. 
</div>

# Regression Splines
Suppose you want to approximate the function $x(t)$ for some process that you have sampled over the input domain (usually time). If the process isn't linear, then directly using multiple linear regression may not be a good idea. Instead, you can better model the curvature of the process using a regression spline.

With a regression spline, the function is modeled as a linear combination of basis functions:

$$ f(t) = \sum_i c_i \phi_i(t) . $$

While many different types of basis functions exist (e.g., polynomial, Fourier, etc.), it is common to use cubic [B-spline](https://en.wikipedia.org/wiki/B-spline#Definition) basis functions due to their appealing geometric properties, as discussed in [Patrikalakis & Maekawa (2009)](http://web.mit.edu/hyperbook/Patrikalakis-Maekawa-Cho/node17.html). The remainder of this notebook demonstrates how to reformulate a regression spline model as a multiple linear regression in order to approximate a function for the number (in thousands) of Australian residents measured quarterly from March 1971 to March 1994. To do this, we are going use the [Quarterly Time Series of the Number of Australian Residents](https://stat.ethz.ch/R-manual/R-devel/library/datasets/html/austres.html) dataset from the R [`datasets`](https://stat.ethz.ch/R-manual/R-devel/library/datasets/html/00Index.html) package. For convenience, a copy of this dataset is provided at http://cobweb.cs.uga.edu/~mec/austres.csv.

First, let's import some packages from ScalaTion:

In [5]:
:silent                        // suppress extra output
import scalation.analytics._   // for regression
import scalation.linalgebra._  // for linear algebra support
import scalation.columnar_db._ // for relational algebra support
:paste -raw BSpline2.scala

Result printing is on.
import scalation.analytics._
import scalation.linalgebra._
import scalation.columnar_db._
Pasting file BSpline2.scala...


Next, let's load in the dataset using a [`Relation`](http://cobweb.cs.uga.edu/~jam/scalation_1.3/scalation_mathstat/target/scala-2.12/api/scalation/relalgebra/Relation$.html) to see what's available:

In [6]:
val url = "http://cobweb.cs.uga.edu/~mec/austres.csv" // dataset url
val rel = Relation(url, "austres", "SDD", 0, ",")     // create relation
::relation rel

url: String = http://cobweb.cs.uga.edu/~mec/austres.csv
0
rel: scalation.columnar_db.Relation =
Relation(austres, 0,
WrappedArray(id, time, austres),
VectorS(1,	2,	3,	4,	5,	6,	7,	8,	9,	10,	11,	12,	13,	14,	15,	16,	17,	18,	19,	20,	21,	22,	23,	24,	25,	26,	27,	28,	29,	30,	31,	32,	33,	34,	35,	36,	37,	38,	39,	40,	41,	42,	43,	44,	45,	46,	47,	48,	49,	50,	51,	52,	53,	54,	55,	56,	57,	58,	59,	60,	61,	62,	63,	64,	65,	66,	67,	68,	69,	70,	71,	72,	73,	74,	75,	76,	77,	78,	79,	80,	81,	82,	83,	84,	85,	86,	87,	88,	89)
VectorD(1971.25,	1971.50,	1971.75,	1972.00,	1972.25,	1972.50,	1972.75,	1973.00,	1973.25,	1973.50,	1973.75,	1974.00,	1974.25,	1974.50,	1974.75,	1975.00,	1975.25,	1975.50,	1975.75,	1976.00,	1976.25,	1976.50,	1976.75,	1977.00,	1977.25,	1977.50,	1977.75,	1978.00,	1978.25,	1978.50,	1978.75,	1979.00,	1979.25,	1979.50,	1979.75,	1980.00,	1980.25,	1980.50,	...


id,time,austres
1,1971.25,13067.3
2,1971.5,13130.5
3,1971.75,13198.4
4,1972.0,13254.2
5,1972.25,13303.7
6,1972.5,13353.9
7,1972.75,13409.3
8,1973.0,13459.2
9,1973.25,13504.5
10,1973.5,13552.6


It looks like we want to use the second column (index 1) to represent points in the input domain and the last column (index 2) to represent our response. Let's extract them as vectors:

In [ ]:
val t = rel.toVectorD(1) // input / time point vector
val y = rel.toVectorD(2) // response vector
::plotv y

t: scalation.linalgebra.VectorD = VectorD(1971.25,	1971.50,	1971.75,	1972.00,	1972.25,	1972.50,	1972.75,	1973.00,	1973.25,	1973.50,	1973.75,	1974.00,	1974.25,	1974.50,	1974.75,	1975.00,	1975.25,	1975.50,	1975.75,	1976.00,	1976.25,	1976.50,	1976.75,	1977.00,	1977.25,	1977.50,	1977.75,	1978.00,	1978.25,	1978.50,	1978.75,	1979.00,	1979.25,	1979.50,	1979.75,	1980.00,	1980.25,	1980.50,	1980.75,	1981.00,	1981.25,	1981.50,	1981.75,	1982.00,	1982.25,	1982.50,	1982.75,	1983.00,	1983.25,	1983.50,	1983.75,	1984.00,	1984.25,	1984.50,	1984.75,	1985.00,	1985.25,	1985.50,	1985.75,	1986.00,	1986.25,	1986.50,	1986.75,	1987.00,	1987.25,	1987.50,	1987.75,	1988.00,	1988.25,	1988.50,	1988.75,	1989.00,	1989.25,	1989.50,	1989.75,	1990.00,	1990.25,	1990.50,	1990.75,	1991.00,	1991.25,	1991.50,	1991.75,	1992.00,...
y: scalation.linalgebra.VectorD = VectorD(13067.3,	13130.5,	13198.4,	13254.2,	13303.7,	13353.9,	13409.3,	13459.2,	13504.5,	13552.6,	13614.3,	13669.5,	13722.6,	13772.1,	13832.0,	13862.6,	13893.0,	1392

## Generating B-spline Basis Functions
Now, let's prepare for the creation of our B-spline basis functions. The degree of a B-spline basis function is one less than its order. Therefore, we'll use $k=4$ to denote the order for a cubic B-spline. Furthermore, B-spline basis functions are parameterized according to a vector of non-decreasing "knots" across the input domain. The regression spline will only be defined on the closed interval defined by the first and last elements of the knot vector. As described in [Patrikalakis, N. M., & Maekawa, T. (2009)](http://web.mit.edu/hyperbook/Patrikalakis-Maekawa-Cho/node17.html), it is often desirable to "clamp" the knot vector by repeating the first and last time points $k$ times to ensure that the resulting regression spline passes through the first and last points in the response. **<span style="color:blue">NOTE:</span>** We encourage the user to try the remaining steps in this notebook with different sets of internal knots (keeping the endpoints the same).

In [8]:
val k     = 4                        // order 4; degree 3
val knots = BSpline2Util.clamp(k, t) // clamped knot vector
val bs    = new BSpline2(knots, k)   // cubic b-spline basis generator

k: Int = 4
knots: scalation.linalgebra.VectorD = VectorD(1971.25,	1971.25,	1971.25,	1971.25,	1971.50,	1971.75,	1972.00,	1972.25,	1972.50,	1972.75,	1973.00,	1973.25,	1973.50,	1973.75,	1974.00,	1974.25,	1974.50,	1974.75,	1975.00,	1975.25,	1975.50,	1975.75,	1976.00,	1976.25,	1976.50,	1976.75,	1977.00,	1977.25,	1977.50,	1977.75,	1978.00,	1978.25,	1978.50,	1978.75,	1979.00,	1979.25,	1979.50,	1979.75,	1980.00,	1980.25,	1980.50,	1980.75,	1981.00,	1981.25,	1981.50,	1981.75,	1982.00,	1982.25,	1982.50,	1982.75,	1983.00,	1983.25,	1983.50,	1983.75,	1984.00,	1984.25,	1984.50,	1984.75,	1985.00,	1985.25,	1985.50,	1985.75,	1986.00,	1986.25,	1986.50,	1986.75,	1987.00,	1987.25,	1987.50,	1987.75,	1988.00,	1988.25,	1988.50,	1988.75,	1989.00,	1989.25,	1989.50,	1989.75,	1990.00,	1990.25,	1990.50,	1990.75,	1991.00,	1991...
bs: BSpline2 = BSpline2@4c40aed1


Now we are ready to define $\phi_i(t)$:

In [9]:
val nsf  = bs.count()                     // number of spline basis functions (for i)
def phi (i: Int)(t: Double) = bs(k)(i)(t) // the basis function
val step = 5E-4                           // step size for plotting 

nsf: Int = 91
phi: (i: Int)(t: Double)Double
step: Double = 5.0E-4


Let's take a look at the first five B-spline basis functions along the sub-interval defined by $[t_0, t_5]$:

In [ ]:
def bf(i: Int) = VectorD(Range.BigDecimal(t(0), t(5), step).map(_.toDouble).map(phi(i)).toSeq) // evaluate basis function i along the interval
val bfs = MatrixD((0 until 5).map(bf).toSeq)                                                   // create matrix with a column for each function
::plotm bfs

bf: (i: Int)scalation.linalgebra.VectorD
bfs: scalation.linalgebra.MatrixD =

MatrixD(1.00000,	0.00000,	0.00000,	0.00000,	0.00000,
	0.994012,	0.00598201,	5.99267e-06,	1.33333e-09,	0.00000,
	0.988048,	0.0119281,	2.39413e-05,	1.06667e-08,	0.00000,
	0.982108,	0.0178384,	5.38020e-05,	3.60000e-08,	0.00000,
	0.976191,	0.0237129,	9.55307e-05,	8.53333e-08,	0.00000,
	0.970299,	0.0295518,	0.000149083,	1.66667e-07,	0.00000,
	0.964430,	0.0353550,	0.000214416,	2.88000e-07,	0.00000,
	0.958585,	0.0411228,	0.000291485,	4.57333e-07,	0.00000,
	0.952764,	0.0468552,	0.000380245,	6.82667e-07,	0.00000,
	0.946966,	0.0525522,	0.000480654,	9.72000e-07,	0.00000,
	0.941192,	0.0582140,	0.000592667,	1.33333e-06,	0.00000,
	0.935441,	0.0638406,	0.000716239,	1.77467e-06,	0.00000,
	0.929714,	0.0694322,	0.000851328,	2.30400e-06,	0.00000,
	0.924010,	0.0749888,	...


As you can see, the first two basis functions (and the last two, although unseen) seem exhibit more acceleration than the rest. This is verified by looking at their second derivatives along the same subinterval:

In [ ]:
def d2phi(i: Int)(t: Double) = bs.derivative(k, 2, t)(i)
def d2bf(i: Int) = VectorD(Range.BigDecimal(t(0), t(5), step).map(_.toDouble).map(d2phi(i)).toSeq)
val d2bfs = MatrixD((0 until 5).map(d2bf).toSeq)
::plotm d2bfs

d2phi: (i: Int)(t: Double)Double
d2bf: (i: Int)scalation.linalgebra.VectorD
d2bfs: scalation.linalgebra.MatrixD =

MatrixD(0.00000,	0.00000,	0.00000,	0.00000,	0.00000,
	0.00000,	-107.736,	35.8480,	0.0320000,	0.00000,
	0.00000,	-107.472,	35.6960,	0.0640000,	0.00000,
	0.00000,	-107.208,	35.5440,	0.0960000,	0.00000,
	0.00000,	-106.944,	35.3920,	0.128000,	0.00000,
	0.00000,	-106.680,	35.2400,	0.160000,	0.00000,
	0.00000,	-106.416,	35.0880,	0.192000,	0.00000,
	0.00000,	-106.152,	34.9360,	0.224000,	0.00000,
	0.00000,	-105.888,	34.7840,	0.256000,	0.00000,
	0.00000,	-105.624,	34.6320,	0.288000,	0.00000,
	0.00000,	-105.360,	34.4800,	0.320000,	0.00000,
	0.00000,	-105.096,	34.3280,	0.352000,	0.00000,
	0.00000,	-104.832,	34.1760,	0.384000,	0.00000,
	0.00000,	-104.568,	34.0240,	0.416000,	0.00000,
	0.00000,	-104.304,	33.8720,	0.448000,	0.00000,
	0.00000,	-104.040,	33.7200,	0...


Let's keep thos accerlation in mind for later. It's going to have an impact on the regression.

## Ordinary Least Squares Method 
To reformulate the regression spline model as a multiple linear regression model, we need to construct a design matrix using the basis functions. Each column in the matrix will represent one of the basis functions. Each row will represent each of the basis functions evaluated at a particular input point in the closed interval defined by the knot vector. Then, we have:

$$ f(\mathbf{t}) = X\mathbf{c}, $$

where $\mathbf{t}$ is the vector of input points and $\mathbf{c}$ is a coefficient vector to be estimated. Now suppose we have a set of samples for $f(t)$ over the input points:

$$ \mathbf{y} = f(\mathbf{t}) + \mathbf{e}, $$

where $\mathbf{y}$ is the response vector and $\mathbf{e}$ is the error not explained by the model. This allows us to derive a formulation for $\mathbf{e}$ in terms of $X$ and $\mathbf{c}$:

$$ \mathbf{y} = f(\mathbf{t}) + \mathbf{e}, $$

$$ \mathbf{y} = f(\mathbf{t}) + \mathbf{e} = X\mathbf{c} + \mathbf{e}, $$

$$ \mathbf{y} = X\mathbf{c} + \mathbf{e}, $$

$$ \mathbf{e} = \mathbf{y} - X\mathbf{c}. $$

Now that we have a formulation for the error, let's find an estimate for $\mathbf{c}$ that minimizes the *sum of squared error*  (i.e., $\mathbf{e}'\mathbf{e}$). This can be solved using the ordinary least squares (OLS) method:

$$ D^1_\mathbf{c}[\mathbf{e}'\mathbf{e} ] = 0$$

$$ D^1_\mathbf{c}[(\mathbf{y} - X\mathbf{c})'(\mathbf{y} - X\mathbf{c}) ] = 0$$

$$ -2X'\mathbf{y} + 2X'X\mathbf{c} = 0$$

We can divide both sides by two:

$$ -X'\mathbf{y} + X'X\mathbf{c} = 0$$

$$ X'X\mathbf{c} =  X'\mathbf{y}$$

$$ X'X \mathbf{c} =  X'\mathbf{y}$$

$$ \hat{\mathbf{c}} = (X'X)^{-1}X'\mathbf{y} $$

Let's create the design matrix and solve for the coefficient vector using the OLS method:

In [12]:
val X = new MatrixD (t.dim, bs.count())                   // create empty design matrix
for (j <- X.range1; i <- X.range2) X(j, i) = phi(i)(t(j)) // evaluate each spline at each input point
//val X = bs(k, t) // pending bug fix from Dr. Miller
val c = (X.t * X).inverse * X.t * y                       // solve for x using OLS

X: scalation.linalgebra.MatrixD =

MatrixD(0.00000,	0.00000,	0.00000,	0.00000,	0.00000,	0.00000,	0.00000,	0.00000,	0.00000,	0.00000,	0.00000,	0.00000,	0.00000,	0.00000,	0.00000,	0.00000,	0.00000,	0.00000,	0.00000,	0.00000,	0.00000,	0.00000,	0.00000,	0.00000,	0.00000,	0.00000,	0.00000,	0.00000,	0.00000,	0.00000,	0.00000,	0.00000,	0.00000,	0.00000,	0.00000,	0.00000,	0.00000,	0.00000,	0.00000,	0.00000,	0.00000,	0.00000,	0.00000,	0.00000,	0.00000,	0.00000,	0.00000,	0.00000,	0.00000,	0.00000,	0.00000,	0.00000,	0.00000,	0.00000,	0.00000,	0.00000,	0.00000,	0.00000,	0.00000,	0.00000,	0.00000,	0.00000,	0.00000,	0.00000,	0.00000,	0.00000,	0.00000,	0.00000,	0.00000,	0.00000,	0.00000,	0.00000,	0.00000,	0.00000,	0.00000,	0.00000,	0.00000,	0.00000,	0.00000,	0.00000,	0.00000,	0.00000,	0.00000,	0.00000...
c: scalation.linalgebra.VectorD = VectorD(13067.3,	-113232,	-180835,	66998.2,	32698.7,	15469.6,	12541.3,	13653.4,	13424.1,	13526.3,	13541.9,	13617.9,	13668.7,	13725.2,	13766.5,	13841.3,	13860.2,	1389

Before we define $f(t)$ using the regression spline, let's look at the fit of the multiple linear regression model:

In [ ]:
val z   = X * c   // predicted response
val e   = y - z   // residual
val sse = e dot e // sum of squared error
::plotv y z
println(s"SSE = $sse")

z: scalation.linalgebra.VectorD = VectorD(13067.3,	-122629,	19976.1,	35543.7,	17853.0,	13214.7,	13429.9,	13479.3,	13511.9,	13552.0,	13613.7,	13669.6,	13722.7,	13772.1,	13832.0,	13862.6,	13893.0,	13926.8,	13968.9,	14004.7,	14033.1,	14066.0,	14110.1,	14155.6,	14192.2,	14231.7,	14281.5,	14330.3,	14359.3,	14396.6,	14430.8,	14478.4,	14515.7,	14554.9,	14602.5,	14646.4,	14695.4,	14746.6,	14807.4,	14874.4,	14923.3,	14988.7,	15054.1,	15121.7,	15184.2,	15239.3,	15288.9,	15346.2,	15393.5,	15439.0,	15483.5,	15531.5,	15579.4,	15628.5,	15677.3,	15736.7,	15788.3,	15839.7,	15900.6,	15961.5,	16018.3,	16076.9,	16139.0,	16203.0,	16263.3,	16327.9,	16398.9,	16478.3,	16538.2,	16621.6,	16697.0,	16777.2,	16833.1,	16891.6,	16956.8,	17026.3,	17085.4,	17106.9,	17169.3,	17238.9,	17288.3,	17353.9,	17526.6,	17674.1,...
e: scalation.linalgebra.VectorD = VectorD(0.00000,	135759,	-6777.73,	-22289.5,	-4549.34,	139.190,	-20.5579,	-20.1367,	-7.37119,	0.624318,	0.578037,	-0.134696,	-0.0739386,	0.00411652,	-0.00964771,	-0.

Oh no! What happened? The sum of squared error (SSE) is huge. It looks like most of the fit is good, including the end points (as discussed above, due to clamping the knot vector), however, the fit along the subintervals approaching each end point appear to be extremely poor. This is due to the acceleration of the first two and last two basis functions that we observed. The solution? Let's fit the model in such a way as to penalize lots of acceleration. To do this, we use a technique called *smoothing*!

## Smoothing Splines

Roughly speaking, a smoothing spline is a regularized regression spline with a penalty based on the second derivative of the basis functions. If the current sequence of basis functions is used to approximate the position of the function at each input, then the second derivative should indicate how fast the curve is accellerating.

Instead of simply minimizing $\mathbf{e}'\mathbf{e}$ like we did before, we'll minimize $\mathbf{e}'\mathbf{e} + \lambda \lvert\lvert \mathbf{c} \cdot D^2\phi  \rvert\rvert^2_2$. If we recall that the predicted response is estimated by $X \mathbf{c}$, then it's clear that this criterion explodes whenever the basis functions exhibit a large amount of acceleration. 

While criterion certainly looks nice in terms of expression, it easier to work with $D^2\phi$ in matrix form. To do this, construct a matrix such that each column represents the second derivative of a basis function and each row represents the evaluation of the second derivatives at a particular input point. Once we have that matrix, we can square it to get the norm. Let $P$ represent this norm.

In [14]:
val d2bs = new MatrixD(X.dim1, X.dim2)
for (j <- d2bs.range1) d2bs(j) = bs.derivative(k, 2, t(j))
val P    = d2bs.t * d2bs

d2bs: scalation.linalgebra.MatrixD =

MatrixD(0.00000,	0.00000,	0.00000,	0.00000,	0.00000,	0.00000,	0.00000,	0.00000,	0.00000,	0.00000,	0.00000,	0.00000,	0.00000,	0.00000,	0.00000,	0.00000,	0.00000,	0.00000,	0.00000,	0.00000,	0.00000,	0.00000,	0.00000,	0.00000,	0.00000,	0.00000,	0.00000,	0.00000,	0.00000,	0.00000,	0.00000,	0.00000,	0.00000,	0.00000,	0.00000,	0.00000,	0.00000,	0.00000,	0.00000,	0.00000,	0.00000,	0.00000,	0.00000,	0.00000,	0.00000,	0.00000,	0.00000,	0.00000,	0.00000,	0.00000,	0.00000,	0.00000,	0.00000,	0.00000,	0.00000,	0.00000,	0.00000,	0.00000,	0.00000,	0.00000,	0.00000,	0.00000,	0.00000,	0.00000,	0.00000,	0.00000,	0.00000,	0.00000,	0.00000,	0.00000,	0.00000,	0.00000,	0.00000,	0.00000,	0.00000,	0.00000,	0.00000,	0.00000,	0.00000,	0.00000,	0.00000,	0.00000,	0.00000,	0.00...
P: scalation.linalgebra.MatrixD =

MatrixD(0.00000,	0.00000,	0.00000,	0.00000,	0.00000,	0.00000,	0.00000,	0.00000,	0.00000,	0.00000,	0.00000,	0.00000,	0.00000,	0.00000,	0.00000,	0.00000,	0.00000,	0.0

Now we are ready to find an estimate for $\mathbf{c}$ that minimizes the *penalized sum of squared error* criterion (i.e., $\mathbf{e}'\mathbf{e} + \lambda \lvert\lvert \mathbf{c} \cdot D^2\phi  \rvert\rvert^2_2 = \mathbf{e}'\mathbf{e} + \lambda \mathbf{c}' P \mathbf{c}$. This can be done by taking the derivative of the criterion, setting it equal to zero, and solving for $\mathbf{c}$:

$$ D^1_\mathbf{c}[\mathbf{e}'\mathbf{e} + \lambda \mathbf{c}' P \mathbf{c} ] = 0$$

$$ D^1_\mathbf{c}[(\mathbf{y} - X\mathbf{c})'(\mathbf{y} - X\mathbf{c}) + \lambda \mathbf{c}' P \mathbf{c} ] = 0$$

$$ -2X'\mathbf{y} + 2X'X\mathbf{c} + 2 \lambda P \mathbf{c} = 0$$

We can divide both sides by two:

$$ -X'\mathbf{y} + X'X\mathbf{c} + \lambda P \mathbf{c}= 0$$

$$ X'X\mathbf{c} + \lambda P \mathbf{c} =  X'\mathbf{y}$$

$$ (X'X + \lambda P) \mathbf{c} =  X'\mathbf{y}$$

$$ \hat{\mathbf{c}} = (X'X + \lambda P)^{-1}X'\mathbf{y} $$

Let us now estimate $\mathbf{c}$ using the estimator we just derived, letting $\lambda=1$ to start, and predict the response in order to calculate the SSE:

In [ ]:
val lambda = 1.0
val cPen   = ((X.t * X) + (P * lambda)).inverse * X.t * y // use roughness penalty
val zPen   = X * cPen      // predicted response
val ePen   = y - zPen      // residual
val ssePen = ePen dot ePen // sum of squared error
::plotv y zPen
println(s"SSE = $ssePen")

lambda: Double = 1.0
cPen: scalation.linalgebra.VectorD = VectorD(13067.3,	13113.2,	13148.3,	13201.0,	13253.7,	13306.2,	13358.4,	13410.3,	13461.9,	13512.8,	13563.2,	13612.7,	13661.3,	13708.6,	13754.7,	13799.4,	13842.6,	13884.5,	13925.2,	13965.0,	14004.1,	14042.6,	14080.9,	14119.1,	14157.5,	14196.0,	14234.8,	14274.0,	14313.7,	14353.9,	14394.9,	14436.8,	14479.8,	14524.1,	14569.8,	14617.1,	14666.1,	14716.7,	14768.9,	14822.5,	14877.3,	14933.0,	14989.3,	15045.8,	15102.3,	15158.3,	15213.8,	15268.4,	15322.2,	15375.3,	15427.7,	15479.7,	15531.5,	15583.3,	15635.5,	15688.2,	15741.8,	15796.3,	15852.0,	15908.9,	15967.2,	16026.9,	16088.0,	16150.5,	16214.4,	16279.5,	16345.7,	16412.8,	16480.4,	16548.4,	16616.3,	16683.7,	16750.3,	16815.8,	16879.9,	16942.6,	17003.8,	17063.3,	17121.4,	17178.0,	17233.3,	17287.3,	17340.2,	17392...
zPen: scalation.linalgebra.VectorD = VectorD(13067.3,	13148.3,	13201.0,	13253.6,	13306.1,	13358.4,	13410.3,	13461.8,	13512.7,	13563.0,	13612.6,	13661.1,	13708.4,	13754.5,	13799.1

Well, the fit not only looks better, it is better as evidenced by the smaller SSE. Can we make it even better? Consider the assumption we made of $\lambda=1$. Is there a different value that improves the SSE? Let's create a function that computes the SSE for a particular $\lambda$ value:

In [16]:
def tryLambda(lambda: Double): Double = {
    val c   = ((X.t * X) + (P * lambda)).inverse * X.t * y // use roughness penalty
    val z   = X * c   // predicted response
    val e   = y - z   // residual
    e dot e           // return the sum of squared error
} // tryLambda

tryLambda: (lambda: Double)Double


Now that we have a function that can be used for any $\lambda$, let's try repeatedly decreasing its value so long as it improves the SSE for the fit. While there are many different optimization techniques that could be used to find the optimal lambda my minimizing the objective function defined by `tryLambda`, we'll use a naive technique of repeated division for illustrative purposes:

In [17]:
var lambda  = 1.0
var sse     = Double.PositiveInfinity
var ssePrev = Double.PositiveInfinity

do {
  lambda  = lambda / 10       // make lambda smaller
  ssePrev = sse               // save previous SSE
  sse     = tryLambda(lambda) // compute new SSE
  println(s"lambda = $lambda; sse = $sse")
} while (sse < ssePrev)       // loop while SSE improves

lambda = lambda * 10          // save the best SSE

lambda: Double = 1.0
sse: Double = Infinity
ssePrev: Double = Infinity

lambda = 0.1; sse = 4529.381700638758
lambda = 0.01; sse = 2069.1094806884958
lambda = 0.001; sse = 917.5327455827379
lambda = 1.0E-4; sse = 217.49209757188095
lambda = 1.0E-5; sse = 14.153205610632988
lambda = 1.0000000000000002E-6; sse = 0.21739436372478965
lambda = 1.0000000000000002E-7; sse = 0.0022862403434570993
lambda = 1.0000000000000002E-8; sse = 2.2979956262516784E-5
lambda = 1.0000000000000003E-9; sse = 2.2991389344649151E-7
lambda = 1.0000000000000003E-10; sse = 2.2995986655720284E-9
lambda = 1.0000000000000003E-11; sse = 1.5583188050194268E-10
lambda = 1.0000000000000002E-12; sse = 8.343089209034168E-9

lambda: Double = 1.0000000000000001E-11


Let's observe the fit using this new, optimal $\lambda$ value: 

In [ ]:
val cPen2   = ((X.t * X) + (P * lambda)).inverse * X.t * y // use roughness penalty
val zPen2   = X * cPen2       // predicted response
val ePen2   = y - zPen2       // residual
val ssePen2 = ePen2 dot ePen2 // sum of squared error
::plotv y zPen2
println(s"lambda = $lambda; sse = $ssePen2")

cPen2: scalation.linalgebra.VectorD = VectorD(13067.3,	13082.0,	13131.2,	13201.0,	13255.1,	13303.7,	13352.2,	13410.9,	13459.9,	13504.7,	13548.4,	13617.2,	13668.7,	13725.1,	13766.6,	13841.3,	13860.2,	13893.4,	13924.2,	13970.7,	14006.3,	14032.2,	14063.5,	14109.7,	14158.1,	14191.4,	14229.5,	14280.7,	14336.5,	14354.9,	14399.8,	14425.6,	14482.5,	14514.8,	14552.5,	14604.5,	14644.5,	14695.8,	14744.8,	14804.6,	14881.2,	14917.0,	14990.6,	15052.7,	15123.1,	15185.2,	15241.2,	15285.6,	15349.7,	15392.9,	15439.7,	15482.3,	15532.0,	15578.7,	15629.7,	15673.6,	15739.6,	15788.2,	15837.3,	15900.9,	15962.6,	16017.7,	16076.3,	16138.3,	16204.4,	16261.9,	16327.7,	16394.8,	16486.5,	16528.9,	16627.1,	16692.5,	16785.0,	16830.7,	16890.7,	16956.1,	17025.9,	17098.3,	17093.4,	17169.6,	17244.6,	17288.4,	17353.8,	1742...
zPen2: scalation.linalgebra.VectorD = VectorD(13067.3,	13130.5,	13198.4,	13254.2,	13303.7,	13353.9,	13409.3,	13459.2,	13504.5,	13552.6,	13614.3,	13669.5,	13722.6,	13772.1,	13832.0,	13862.6,	13893.0,	

That's way better! The SSE is really good. We can now, finally and confidently, define $f(t)$ using the coefficient vector that we estimated for the smoothing spline:

In [19]:
def f(t: Double) = (0 until bs.count()).map(i => c(i) * phi(i)(t)).sum 

f: (t: Double)Double


Now you should be able to use $f(t)$ to compute values anywhere along closed interval defined by the ends of the original knot vector.

## References
* Patrikalakis & Maekawa (2009). *Shape Interrogation for Computer Aided Design and Manufacturing.* Springer Science & Business Media.
* Brockwell & Davis (1996). *Introduction to Time Series and Forecasting.* Springer.